In this exercise, we'll be relating learning to access information about the environment or landscape in a raster format. We'll then be relating the locations of visible sites to various environmental factors. 

Environmental data is often held in a raster format because the data is continuous. Elevation is everywhere, right? So if you have a big image file (a raster) you can have a value for elevation in pretty much every pixel of that file. 

So far in this course you've been working mostly with vector data (points, lines, polygons) but now you're going to work with rasters as well.

As usual start by getting some tools.

In [0]:
!pip install rasterio
!pip install elevation
!pip install richdem

In [0]:

import rasterio     # import the main rasterio function
import matplotlib   # matplotlib is the primary python plotting and viz library
# this bit of magic allows matplotlib to plot inline in a jupyter notebook
%matplotlib inline  

# We can check which version we're running by printing the "__version__" variable
print("rasterio's version is: " + rasterio.__version__)
print(rasterio)

Start by reading in a raster dataset in the form of an image.

In [0]:
# filepath to our image
img_fp = 'http://ropitz.github.io/digitalantiquity/data/LE70220492002106EDC00_stack.gtif'

# Open a geospatial dataset
dataset = rasterio.open(img_fp)
print(dataset)

Let's learn some basic things about the image data

In [0]:

# what is the name of this image
img_name = dataset.name
print('Image filename: {n}\n'.format(n=img_name))

# How many bands does this image have?
num_bands = dataset.count
print('Number of bands in image: {n}\n'.format(n=num_bands))

# How many rows and columns?
rows, cols = dataset.shape
print('Image size is: {r} rows x {c} columns\n'.format(r=rows, c=cols))

# Does the raster have a description or metadata?
desc = dataset.descriptions
metadata = dataset.meta

print('Raster description: {desc}\n'.format(desc=desc))

# What driver was used to open the raster?
driver = dataset.driver
print('Raster driver: {d}\n'.format(d=driver))

# What is the raster's projection?
proj = dataset.crs
print('Image projection:')
print(proj, '\n')

# What is the raster's "geo-transform"
gt = dataset.transform

print('Image geo-transform:\n{gt}\n'.format(gt=gt))

print('All raster metadata:')
print(metadata)
print('\n')

The first few pieces of information we obtained are fairly straightforward - image name, the raster size, the number of bands, a description, some metadata, and the raster's file format.

The image's projection is formatted in what's known as "Well Known Text". For more information on specific projections and for format conversions among projection description formats (e.g., proj4 string, WKT, ESRI WKT, JSON, etc.) see Spatial Reference.

The last piece of information we accessed is something called a "geotransform". This set of 6 numbers provides all the information required to and from transform pixel and projected coordinates. In this example, the first number (462405) and the fourth number (1741815) are the top left of the upper left pixel of the raster. The pixel size in x and y dimensions of the raster is listed as the second (30) and the sixth (-30) numbers. Since our raster is north up oriented, the third and fifth numbers are 0. 

---



# **Image raster bands**
now for the fun part, actually visualizing and working with the data
The rasterio Dataset object we created contains a lot of useful information but it is not directly used to read in the raster image. Instead we will need to access the raster's bands using the read() method:

In [0]:
# Open the fourth band in our image - NIR here
nir = dataset.read(4)
nir.shape # check out the dimensions of the image

In [0]:
#get numpy
import numpy as np

In [0]:
# What are the band's datatypes?
datatype = dataset.dtypes
print('Band datatypes: {dt}'.format(dt=datatype))

# How about some band statistics?
band_mean = np.mean(nir)
band_min = np.amin(nir)
band_max = np.amax(nir)
band_stddev = np.std(nir)
print('Band range: {minimum} - {maximum}'.format(maximum=band_max,
                                                 minimum=band_min))
print('Band mean, stddev: {m}, {s}\n'.format(m=band_mean, s=band_stddev))

In [0]:
#read in the image and display it
full_img = dataset.read()
full_img.shape # bands, rows, cols
from rasterio.plot import show 
# import the show function which allows us to display the image

print("Image dimensions: ", full_img.shape)
show(nir, transform=dataset.transform, cmap='gray')

In [0]:
#more tools
import matplotlib   # matplotlib is the primary python plotting and viz library
import matplotlib.pyplot as plt

# this bit of magic allows matplotlib to plot inline in a jupyter notebook
%matplotlib inline  
import folium       # folium is an interactive mapping library


Some raster datasets will only have one band. These behave a bit like a black and white image. Rasters can contain as many bands as you like, and single band images can be combined into multiple band images. Multiple band images are useful because you can calculate ratios between the different bands to enhance the visibility of certain features in your image.

In [0]:
# to make a multiple band raster add all the file paths to a list
s2_bands = [
            "http://ropitz.github.io/digitalantiquity/data/sentinel-2/2018-10-13_Sentinel-2BL1C_B02.tiff",
            "http://ropitz.github.io/digitalantiquity/data/sentinel-2/2018-10-13_Sentinel-2BL1C_B03.tiff",
            "http://ropitz.github.io/digitalantiquity/data/sentinel-2/2018-10-13_Sentinel-2BL1C_B04.tiff",
            "http://ropitz.github.io/digitalantiquity/data/sentinel-2/2018-10-13_Sentinel-2BL1C_B08.tiff"
]

# open these files and add all bands to an array
arrs = []
for band in s2_bands:
    with rasterio.open(band) as f:
        arrs.append(f.read(1))

# convert the list to a numpy array
sentinel_img = np.array(arrs, dtype=arrs[0].dtype)
# let's check the shape of this array
sentinel_img.shape

In [0]:
#clip to get a smaller area and show the multiple band image data
clipped_img = sentinel_img[:, 0:750:, 0:1500]
clipped_img.shape
show(clipped_img[[2,1,0], :, :])

You can see the different information held in each band by plotting a histogram of the raster dataset.

In [0]:
# plot a histogram of the data in each band
rasterio.plot.show_hist(clipped_img, bins=50, histtype='stepfilled', lw=0.0, stacked=False, alpha=0.3)

Even from simple visualizations we can see the contrast between the red and the near-infared (NIR) bands. Also note that the peak at 255 is simply from areas with no data which are labeled 255 on all bands. For example:


In [0]:

clipped_img[:,0,0]

Different band ratios will highlight different features in a raster dataset. We might be interested in finding places where cropmarks are located, for example. There are lots of band ratios that will highlight more green and more stressed vegetation. A common ratio that does this is called NDVI- the normalized difference vegetation index.

In [0]:
#calculate NDVI
# just ignoring this error because the image has lots of NaN pixels
np.seterr(divide='ignore', invalid='ignore')

bandNIR = clipped_img[3] # fourth band
bandRed = clipped_img[2] # second band

# note that in python division of integers leads to integers so we need to specify floats in order to get floats
ndvi = (bandNIR.astype(float)-bandRed.astype(float))/(bandNIR.astype(float)+bandRed.astype(float))

In [0]:
#show NDVI
plt.imshow(ndvi, cmap="RdYlGn")
plt.colorbar()
plt.show()

In this second part of the exercise, we'll be calculating some properties of the landscape like slope and aspect that are commonly used in analyses and models of where sites are likely to be visible


In [0]:
!pip install geopandas

In [0]:
#more tools. all the tools really
from shapely.geometry import mapping
from shapely.geometry import Point
from rasterio.mask import mask
import geopandas as gpd
import zipfile
import io
import requests
import elevation
import richdem as rd
import gdal

And now get some data to start the second part of the exercise- a raster dataset representing elevation referred to as a DEM or 'digital elevation model'.

In [0]:
#fetch elevation data from the SRTM server and clip to our area of interest
dem_path = os.path.join(os.getcwd(), 'areaDEM.tif')
elevation.clip(bounds=(5.1, 43.65, 5.5, 43.95), output=dem_path)

In [0]:
#check the DEM has loaded nicely by plotting it
areaDEM = rd.LoadGDAL(dem_path)
plt.imshow(areaDEM, interpolation='none')
plt.colorbar()
plt.show()

Compute and plot slope and aspect

You can use the rd.TerrainAttribute function to compute slope and aspect for each pixel. Note that there are multiple ways to represent the slope values. Read the richdem docs for more options.

To visualize slope and aspect, you can use the rdShow function.

In [0]:
slope = rd.TerrainAttribute(areaDEM, attrib='slope_riserun')
rd.rdShow(slope, axes=False, cmap='magma', figsize=(8, 5.5))
plt.show()

In [0]:
# now do the same thing to calculate and plot the aspect data
aspect = rd.TerrainAttribute(areaDEM, attrib='aspect')
rd.rdShow(aspect, axes=False, cmap='jet', figsize=(8, 5.5))
plt.show()

Contours, a common way to visualize elevation data, can also be derived from a raster.

In [0]:
#open the raster file using gdal
gdal_data = gdal.Open(dem_path)
gdal_band = gdal_data.GetRasterBand(1)
nodataval = gdal_band.GetNoDataValue()

# convert to a numpy array
data_array = gdal_data.ReadAsArray().astype(np.float)
data_array

# replace missing values if necessary
if np.any(data_array == nodataval):
    data_array[data_array == nodataval] = np.nan

In [0]:
#Plot out data with Matplotlib's 'contour'
fig = plt.figure(figsize = (12, 8))
ax = fig.add_subplot(111)
plt.contour(data_array, cmap = "viridis", 
            levels = list(range(0, 5000, 100)))
plt.title("Elevation Contours")
cbar = plt.colorbar()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [0]:
#Plot our data with Matplotlib's 'contourf' to get filled contours
fig = plt.figure(figsize = (12, 8))
ax = fig.add_subplot(111)
plt.contourf(data_array, cmap = "viridis", 
            levels = list(range(0, 5000, 500)))
plt.title("Elevation Contours")
cbar = plt.colorbar()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

That's it for this lesson. Hopefully you've seen how you can access and manipulate raster data to represent the landscape or environmental factors. 